In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init
from torch.utils.data import DataLoader
import torch.optim as optim

In [2]:
learning_rate=0.001
epochs=30
batch_size=256

In [3]:
mnist_train=dsets.MNIST(root='data/MNIST_data',
                        train=True,
                        transform=transforms.ToTensor(),
                        download=True)
mnist_test=dsets.MNIST(root='data/MNIST_data',
                        train=False,
                        transform=transforms.ToTensor(),
                        download=True)

100%|██████████| 9912422/9912422 [00:00<00:00, 34195951.37it/s]


Extracting data/MNIST_data\MNIST\raw\train-images-idx3-ubyte.gz to data/MNIST_data\MNIST\raw



100%|██████████| 28881/28881 [00:00<00:00, 328341.52it/s]


Extracting data/MNIST_data\MNIST\raw\train-labels-idx1-ubyte.gz to data/MNIST_data\MNIST\raw



100%|██████████| 1648877/1648877 [00:00<00:00, 2610144.62it/s]


Extracting data/MNIST_data\MNIST\raw\t10k-images-idx3-ubyte.gz to data/MNIST_data\MNIST\raw



100%|██████████| 4542/4542 [00:00<?, ?it/s]

Extracting data/MNIST_data\MNIST\raw\t10k-labels-idx1-ubyte.gz to data/MNIST_data\MNIST\raw



In [4]:
print(mnist_test)
print(mnist_train)

Dataset MNIST
    Number of datapoints: 10000
    Root location: data/MNIST_data
    Split: Test
    StandardTransform
Transform: ToTensor()
Dataset MNIST
    Number of datapoints: 60000
    Root location: data/MNIST_data
    Split: Train
    StandardTransform
Transform: ToTensor()


In [5]:
data_loader=DataLoader(dataset=mnist_train,
                       batch_size=batch_size,
                       shuffle=True,
                       drop_last=True)
test_loader=DataLoader(dataset=mnist_test,
                       batch_size=batch_size,
                       shuffle=True,
                       drop_last=True)

In [6]:
for X, Y in data_loader:
  print(X.size(), Y.size())
  break

torch.Size([256, 1, 28, 28]) torch.Size([256])


In [7]:
cnt_batch_train=len(data_loader)
cnt_batch_test=len(test_loader)
print(cnt_batch_train, cnt_batch_test)

234 39


In [8]:
class CNN(nn.Module):
  def __init__(self):
    super().__init__()

    self.layer1=nn.Sequential(
      nn.Conv2d(1, 32, kernel_size=3, stride=1,padding=1),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2, stride=2) #[32,14,14]
    )
    self.layer2=nn.Sequential(
      nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2, stride=2) #[64,7,7]
    )
    self.fc=nn.Linear(7*7*64, 10, bias=True)
    nn.init.xavier_uniform_(self.fc.weight) # 완전연결층 한정 가중치 초기화

  def forward(self, x):
    out=self.layer1(x)
    out=self.layer2(out)
    out=out.view(out.size(0), -1) #[256,64,7,7]=>[265,64*7*7]
    out=self.fc(out)
    return out

In [9]:
device='cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(777)
if device=='cuda':
    torch.cuda.manual_seed_all(777)

In [10]:
model=CNN().to(device)
crit=nn.CrossEntropyLoss().to(device)
optimizer=optim.Adam(model.parameters(), lr=learning_rate)

In [11]:
for epoch in range(epochs):
  avg_cost=0

  for X, Y in data_loader:
    X=X.to(device)
    Y=Y.to(device)

    y_hat=model(X)
    cost=crit(y_hat, Y)
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    avg_cost+=cost/cnt_batch_train
  print('epoch:{}, cost:{}'.format(epoch, avg_cost.item()))


epoch:0, cost:0.3640304505825043
epoch:1, cost:0.08700936287641525
epoch:2, cost:0.0591961145401001
epoch:3, cost:0.04933690279722214
epoch:4, cost:0.04188058152794838
epoch:5, cost:0.03708373382687569


KeyboardInterrupt: 

In [ ]:
with torch.no_grad():
  accuracys=0

  for X, Y in test_loader:
    X=X.to(device)
    Y=Y.to(device)

    pred=model(X)
    accuracy=(pred.argmax(dim=1)==Y).sum()
    accuracys+=accuracy
    print(pred.argmax(dim=1))
    print(Y)
    print(accuracy)
print(accuracys/(cnt_batch_test*batch_size))

tensor([0, 6, 1, 5, 8, 4, 3, 0, 1, 6, 1, 8, 5, 6, 1, 8, 8, 6, 7, 2, 3, 9, 6, 1,
        8, 4, 5, 2, 5, 6, 9, 9, 6, 4, 1, 0, 6, 9, 5, 2, 9, 6, 2, 3, 8, 0, 7, 4,
        0, 3, 1, 0, 8, 4, 3, 7, 0, 9, 0, 6, 9, 2, 4, 0, 8, 5, 8, 0, 1, 0, 9, 4,
        1, 7, 8, 5, 3, 0, 4, 7, 3, 2, 0, 4, 9, 7, 4, 1, 1, 1, 0, 3, 4, 9, 1, 2,
        5, 9, 7, 9, 2, 0, 4, 1, 6, 8, 7, 1, 5, 4, 3, 1, 7, 0, 9, 8, 7, 9, 7, 8,
        9, 4, 1, 2, 7, 6, 7, 9, 2, 7, 1, 9, 2, 1, 5, 1, 8, 1, 6, 5, 7, 4, 9, 7,
        5, 5, 7, 0, 5, 3, 6, 4, 0, 5, 8, 6, 5, 4, 9, 1, 4, 0, 8, 0, 0, 7, 1, 0,
        5, 4, 4, 6, 1, 1, 2, 6, 7, 1, 8, 8, 2, 8, 4, 8, 6, 6, 5, 0, 8, 4, 1, 5,
        1, 9, 0, 7, 9, 1, 8, 8, 4, 7, 8, 8, 6, 5, 8, 8, 3, 7, 9, 7, 7, 0, 1, 6,
        4, 9, 6, 7, 1, 1, 8, 4, 3, 7, 6, 1, 9, 8, 8, 2, 4, 8, 7, 8, 9, 1, 9, 5,
        4, 7, 4, 5, 6, 2, 2, 3, 0, 0, 2, 9, 0, 3, 9, 3])
tensor([0, 6, 1, 5, 8, 4, 3, 0, 1, 6, 1, 8, 5, 6, 1, 8, 8, 6, 7, 2, 3, 9, 6, 1,
        8, 4, 5, 2, 5, 6, 9, 9, 6, 4, 1, 0, 6, 9, 5, 2, 7, 6, 2

In [ ]:
fasaionmnist_train=dsets.FashionMNIST(root='data/FashionMNIST_data',
                        train=True,
                        transform=transforms.ToTensor(),
                        download=True)
fasaionmnist_test=dsets.MNIST(root='data/FashionMNIST_data',
                        train=False,
                        transform=transforms.ToTensor(),
                        download=True)